# Dakota SNS Workflow
A simple pipeline workflow that runs a Dakota job and a postprocessing job.

This workflow takes for granted that dakota and mantid are installed on the machine under /opt/dakota and /opt/Mantid.

## Import Pegasus API and define directory

In [ ]:
from Pegasus.jupyter.instance import *
workflow_dir = '/local-scratch/georgpap/jypyter_dakota_sns_test'

## Add information about input files to the replica catalog

In [ ]:
rc = ReplicaCatalog(workflow_dir)
rc.add('DS.in', 'https://raw.githubusercontent.com/pegasus-isi/dakota-sns-workflow/master/input/DS.in', site='GitHub')
rc.add('LiCl_290K.nxs', 'https://raw.githubusercontent.com/pegasus-isi/dakota-sns-workflow/master/input/LiCl_290K.nxs', site='GitHub')

## Add information about executables to the transformation catalog

In [ ]:
tc = TransformationCatalog(workflow_dir)
e_DakotaWrapper = Executable('DakotaWrapper', arch=Arch.X86_64, os=OSType.LINUX, installed=False)
e_DakotaWrapper.addPFN(PFN('https://raw.githubusercontent.com/pegasus-isi/dakota-sns-workflow/master/bin/DakotaWrapper.sh', 'condorpool'))

e_DS = Executable('DS.py', arch=Arch.X86_64, os=OSType.LINUX, installed=False)
e_DS.addPFN(PFN('https://raw.githubusercontent.com/pegasus-isi/dakota-sns-workflow/master/bin/DS.py', 'condorpool'))

e_DSout = Executable('DSout', arch=Arch.X86_64, os=OSType.LINUX, installed=False)
e_DSout.addPFN(PFN('https://raw.githubusercontent.com/pegasus-isi/dakota-sns-workflow/master/bin/DSout.py', 'condorpool'))

tc.add(e_DakotaWrapper)
tc.add(e_DS)
tc.add(e_DSout)

## Add sites catalog

In [ ]:
sc = SitesCatalog(workflow_dir)
sc.add_site('condorpool', arch=Arch.X86_64, os=OSType.LINUX)
sc.add_site_profile('condorpool', namespace=Namespace.PEGASUS, key='style', value='condor')
sc.add_site_profile('condorpool', namespace=Namespace.CONDOR, key='universe', value='vanilla')

## Workflow description

In [ ]:
dax = ADAG("dakota_sns_pipeline")

dakota_in = File('DS.in')
sim_data = File('LiCl_290K.nxs')
dakota_out = File('DS.out')
dakota_stdout = File('DS.stdout')
dakota_rst = File('dakota.rst')
DSout_plot = File('DSout.png')
DSout_stdout = File('DSout.stdout')

dakota = Job('DakotaWrapper')
dakota.addArguments('-i', dakota_in, '-o', dakota_out, '-s', dakota_stdout)
dakota.uses(dakota_in, link=Link.INPUT)
dakota.uses(sim_data, link=Link.INPUT)
dakota.uses(e_DS, link=Link.INPUT)
dakota.uses(dakota_out, link=Link.OUTPUT, transfer=True, register=False)
dakota.uses(dakota_stdout, link=Link.OUTPUT, transfer=True, register=False)
dakota.uses(dakota_rst, link=Link.OUTPUT, transfer=True, register=False)
dax.addJob(dakota)

DSout = Job('DSout')
DSout.setStdout(DSout_stdout)
DSout.uses(sim_data, link=Link.INPUT)
DSout.uses(dakota_out, link=Link.INPUT)
DSout.uses(DSout_plot, link=Link.OUTPUT, transfer=True, register=False)
DSout.uses(DSout_stdout, link=Link.OUTPUT, transfer=True, register=False)
dax.addJob(DSout)


## Creating an instance for running the workflow

In [ ]:
instance = Instance(dax, replica_catalog=rc, transformation_catalog=tc, sites_catalog=sc, workflow_dir=workflow_dir)

## Running the workflow

In [ ]:
instance.run(site='condorpool', force=True)

In [ ]:
instance.status(loop=True, delay=5)